In [18]:
import json
import os

In [19]:
data = []
with open('prepared-wikipedia.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if len(l['text']) < 512:
            continue
        data.append(l['text'])

In [20]:
len(data)

96178

In [4]:
# !rm -rf generate-wikipedia-qa
!mkdir generate-wikipedia-qa

mkdir: cannot create directory ‘generate-wikipedia-qa’: File exists


In [5]:
results = []
for i in range(len(data)):
    
    s = """
```
{{replace_me}}
```

return in JSON {
    "name": "QAS",
    "description": "Correctly extracted `QAS` with all the required parameters with correct types",
    "parameters": {
        "$defs": {
            "AnswerEnum": {
                "enum": [
                    "A",
                    "B",
                    "C",
                    "D"
                ],
                "type": "string"
            },
            "Selective_QA": {
                "properties": {
                    "question": {
                        "type": "string"
                    },
                    "A": {
                        "type": "string"
                    },
                    "B": {
                        "type": "string"
                    },
                    "C": {
                        "type": "string"
                    },
                    "D": {
                        "type": "string"
                    },
                    "answer": {
                        "$ref": "#/$defs/AnswerEnum"
                    }
                },
                "required": [
                    "question",
                    "A",
                    "B",
                    "C",
                    "D",
                    "answer"
                ],
                "type": "object"
            }
        },
        "properties": {
            "qa": {
                "items": {
                    "$ref": "#/$defs/Selective_QA"
                },
                "type": "array"
            }
        },
        "required": [
            "qa"
        ],
        "type": "object"
    }
}
    """.strip()
    s = s.replace('{{replace_me}}', data[i])
    results.append(s)
        
len(results)

96178

In [6]:
from openai import OpenAI

def answer(q, i):
    filename = f'generate-wikipedia-qa/{i}.json'
    if os.path.exists(filename):
        return
    
    results = []
    openai = OpenAI(
        base_url='http://localhost:8005/v1',
        api_key='empty',
    )
    for _ in range(3):
    
        
        completion = openai.chat.completions.create(
            model='',
            messages=[{'role': 'user', 'content': q}],
            temperature=0.9,
            max_tokens=2048,
            top_p=0.95,
        )
        r = completion.choices[0].message.content
        results.append(r)
    
    with open(filename, 'w') as fopen:
        json.dump(results, fopen)

In [7]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [8]:
urls = [(q, no) for no, q in enumerate(results)]

In [9]:
answer(*urls[10003])

In [10]:
urls[10003]

('```\nKiteretsu Daihyakka (bahasa Jepun: キテレツ大百科, "Ensiklopedia Kiteretsu") merupakan siri manga sains fiksyen oleh Fujiko F. Fujio yang pernah diterbitkan dalam majalah kanak-kanak "Kodomo no Hikari" dari April 1974 hingga Julai 1977. Manga ini kemudian diadaptasi menjadi siri anime TV sepanjang 331 episod yang eprnah disiarkan dalam rangkaian Fuji TV dari 27 Mac, 1988 hingga 9 Jun, 1996. Anime ini kini disiarkan di Malaysia melalui saluran TV3 sejak Mac 2007. Sekarang siri animasi ini disiarkan setiap Isnin jam 7.30 malam. Watak utamanya adalah kanak-kanak lelaki genius dalam sains bernama Kiteretsu, yang menghasilkan robot sebagai teman bernama Korosuke. Dia kerap bersama rakan-rakannya dan Korosuke dalam mesin cahaya masa yang dibinanya. Miyoko adalah seorang kanak-kanak perempuan dalam kawasan perumahan Kiteretsu yang pokoknya merupakan "teman wanita"nya (sungguhpun mereka masih belajar di sekolah rendah). Terdapat banyak kesamaan dalam pembentukan dan wajah watak-watak utama jik

In [11]:
from threading import Thread
from queue import Queue
from tqdm import tqdm

queue = Queue()
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [12]:
max_worker = 50
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

100%|███████████████████████████████████████████████████████████████████████▉| 96100/96178 [00:01<00:00, 89151.39it/s]

consumer 15 doneconsumer 35 doneconsumer 5 done
consumer 25 done
consumer 16 doneconsumer 4 done



consumer 8 doneconsumer 3 done

consumer 32 done
consumer 12 done
consumer 10 done
consumer 34 doneconsumer 21 done
consumer 31 done
consumer 38 done
consumer 49 done
consumer 46 done

consumer 7 doneconsumer 27 done

consumer 1 done
consumer 42 done
consumer 14 done
consumer 26 done
consumer 44 done
consumer 24 done
consumer 45 done
consumer 36 done
consumer 39 done
consumer 37 done
consumer 29 done
consumer 18 doneconsumer 20 done
consumer 28 doneconsumer 11 done
consumer 47 done


consumer 19 done
consumer 6 done


consumer 23 done
consumer 2 done
consumer 0 doneconsumer 33 done

consumer 48 done
consumer 41 doneconsumer 9 done
consumer 22 done



In [14]:
from glob import glob

files = glob('generate-wikipedia-qa/*.json')
len(files)

96178

In [22]:
files[0]

'generate-wikipedia-qa/77747.json'

In [15]:
with open(files[0]) as fopen:
    d = json.load(fopen)

In [16]:
d

[' {"name": "QAS", "arguments": {"qa": [{"question": "Apakah nama siri anime yang diterangkan dalam paragraph tersebut?", "A": "Pok\\u00e9mon", "B": "Sinnou", "C": "Hikari", "D": "Satoshi", "answer": "A"}, {"question": "Bilakah siri anime tersebut mula disiarkan di Jepun?", "A": "28 September 2006", "B": "27 April 2007", "C": "25 Oktober 2007", "D": "1 Februari 2008", "answer": "A"}, {"question": "Di mana siri anime tersebut disiarkan di Malaysia?", "A": "TV9", "B": "TV3", "C": "Astro", "D": "HBO", "answer": "A"}, {"question": "Siapakah nama koordinator yang ditemui oleh Satoshi dan Takeshi?", "A": "Hikari", "B": "Satoshi", "C": "Hana", "D": "Chihiro", "answer": "A"}, {"question": "Siapakah lagu tema pembuka siri anime tersebut?", "A": "Together", "B": "By Your Side", "C": "Hikari\'s Theme", "D": "Grin", "answer": "A"}]}}',
 ' {"name": "QAS", "arguments": {"qa": [{"question": "Apakah siri anime yang menjadi tumpuan dalam paragraph ini?", "A": "Pok\\u00e9mon", "B": "Naruto", "C": "Death

In [17]:
json.loads(d[0])

{'name': 'QAS',
 'arguments': {'qa': [{'question': 'Apakah nama siri anime yang diterangkan dalam paragraph tersebut?',
    'A': 'Pokémon',
    'B': 'Sinnou',
    'C': 'Hikari',
    'D': 'Satoshi',
    'answer': 'A'},
   {'question': 'Bilakah siri anime tersebut mula disiarkan di Jepun?',
    'A': '28 September 2006',
    'B': '27 April 2007',
    'C': '25 Oktober 2007',
    'D': '1 Februari 2008',
    'answer': 'A'},
   {'question': 'Di mana siri anime tersebut disiarkan di Malaysia?',
    'A': 'TV9',
    'B': 'TV3',
    'C': 'Astro',
    'D': 'HBO',
    'answer': 'A'},
   {'question': 'Siapakah nama koordinator yang ditemui oleh Satoshi dan Takeshi?',
    'A': 'Hikari',
    'B': 'Satoshi',
    'C': 'Hana',
    'D': 'Chihiro',
    'answer': 'A'},
   {'question': 'Siapakah lagu tema pembuka siri anime tersebut?',
    'A': 'Together',
    'B': 'By Your Side',
    'C': "Hikari's Theme",
    'D': 'Grin',
    'answer': 'A'}]}}

In [23]:
data[77747]

'Ini adalah senarai episod untuk yang merupakan siri ke-5 dalam siri anime "Pokémon". Ia asalnya disiarkan di Jepun dari 28 September 2006 hingga 25 Oktober 2007 di TV Tokyo dan di Amerika Syarikat dari 27 April 2007 hingga 1 Februari 2008 di Cartoon Network. Di Malaysia, siri ini dialihsuarakan ke Bahasa Melayu dari versi Jepun dan disiarkan di TV9 bermula 2 Februari 2011 sehingga 10 April 2012. Meliputi pengembaraan berterusan protagonis siri ini Satoshi (disuarakan dalam Bahasa Jepun oleh Rika Matsumoto) dan kawan baiknya Takeshi (disuarakan dalam Bahasa Jepun oleh Yūji Ueda), mereka berdua bertemu seorang koordinator baru bernama Hikari (disuarakan dalam Bahasa Jepun oleh Megumi Toyoguchi), yang mengembara dengan mereka melalui Sinnou dan memasuki Pertandingan Pokemon. Lagu tema pembuka Bahasa Jepun ialah "Together" oleh Fumie Akiyoshi bermula dari episod 4 ketika siaran asal, dan bermula dari episod 2 di DVD. Lagu tema penutup Bahasa Jepun ialah "By Your Side ~Hikari\'s Theme~" (君

In [49]:
data = []
with open('prepared-wikipedia.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if len(l['text']) < 512:
            continue
        data.append(l)

In [77]:
set(q.keys())

{'A', 'B', 'C', 'D', 'answer', 'question'}

In [80]:
suppose = {'question', 'A', 'B', 'C', 'D', 'answer'}
len(set(q.keys()) | suppose)

6

In [116]:
with open('generated-wikipedia-qa.jsonl', 'w') as fopen_j:
    for i in tqdm(range(len(data))):
        f = f'generate-wikipedia-qa/{i}.json'
        if not os.path.exists(f):
            continue
            
        with open(f) as fopen:
            d = json.load(fopen)
        
        questions = []
        unique_questions = set()
        for d_ in d:
            try:
                qa = json.loads(d_)['arguments']['qa']
                for q in qa:
                    if len(set(q.keys()) | suppose) != len(suppose):
                        continue
                    if q['question'].lower() not in unique_questions:
                        if not ('A' in q and isinstance(q['A'], str)):
                            continue
                        if not ('B' in q and isinstance(q['B'], str)):
                            continue
                        if not ('C' in q and isinstance(q['C'], str)):
                            continue
                        if not ('D' in q and isinstance(q['D'], str)):
                            continue
                        if not ('answer' in q and isinstance(q['answer'], str)):
                            continue
                            
                        for k in suppose:
                            q[k] = q[k].encode("utf8", errors="surrogatepass").decode("utf8")

                        questions.append(q)
                        unique_questions.add(q['question'].lower())
            except:
                pass
        
        data[i]['text'] = data[i]['text'].encode("utf8", errors="surrogatepass").decode("utf8")
        data[i]['questions'] = questions
        fopen_j.write(f'{json.dumps(data[i])}\n')

100%|████████████████████████████████████████████████████████████████████████| 96178/96178 [00:07<00:00, 12263.10it/s]


In [117]:
!wc -l generated-wikipedia-qa.jsonl

96178 generated-wikipedia-qa.jsonl


In [118]:
!ls -lh generated-wikipedia-qa.jsonl

-rw-rw-r-- 1 husein husein 433M Mei   9 16:05 generated-wikipedia-qa.jsonl


In [120]:
d = []
with open('generated-wikipedia-qa.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        
        d.append(l)

In [121]:
from datasets import Dataset

In [122]:
dataset = Dataset.from_list(d)

In [124]:
dataset.push_to_hub('mesolitica/mallam-small-malaysian-qa-choice', split = 'wikipedia')

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/97 [00:00<?, ?ba/s]